### RankForecasting-shortterm-paper

test 1k pitmodel

basedon :stint-test-strategy-shortterm
    
+ comparing models: naive, deepar, oracle, pitmodel

In [13]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepstate import DeepStateEstimator
from gluonts.trainer import Trainer
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator, MultivariateEvaluator
from gluonts.distribution.multivariate_gaussian import MultivariateGaussianOutput
from gluonts.model.predictor import Predictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.model.r_forecast import RForecastPredictor
from indycar.model.NaivePredictor import NaivePredictor
from indycar.model.ZeroPredictor import ZeroPredictor
from pandas.plotting import register_matplotlib_converters
from gluonts.model.forecast import SampleForecast
register_matplotlib_converters()

#from indycar.model.stint_predictor_fastrun import *
import indycar.model.stint_simulator_shortterm_pitmodel_savedata as stint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
def simulation(datasetid, testevent, taskid, runts, expid, predictionlen, 
               datamode, loopcnt, featuremode = stint.FEATURE_STATUS,
              pitmodel = 0, model = 'oracle', inlapmode=0):
    #
    # configurataion
    #
    # model path:  <_dataset_id>/<_task_id>-<trainid>/
    #_dataset_id = 'indy2013-2018-nocarid'
    
    stint._inlap_status = inlapmode
    
    stint.init(pitmodel)
    stint._dataset_id = datasetid
    stint._test_event = testevent
    #_test_event = 'Indy500-2019'

    stint._feature_mode = featuremode
    stint._context_ratio = 0.

    stint._task_id = taskid  # rank,laptime, the trained model's task
    stint._run_ts = runts   #COL_LAPTIME,COL_RANK
    stint._exp_id=expid  #rank, laptime, laptim2rank, timediff2rank... 

    stint._train_len = 40
    #stint._debug_carlist = [12]
    stint._test_carlist = [12]
    
    predictor = stint.load_model(predictionlen, model,trainid='2018')

    ret2 = {}
    
    alldata  = []
    for i in range(loopcnt):
        #df, full_samples, full_tss
        ret2[i] = stint.run_simulation_shortterm(predictor, predictionlen, stint.freq, datamode=datamode,
                                                sample_cnt=1)

        data = predictor.prediction_net.savedata
        #savefile = f'RankNet-MLP_savedata_multilayers_b1_s1_t{i}.pickle'
        #with open(savefile, 'wb') as f: 
        #    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
        
        alldata.append(data)
            
        predictor.prediction_net.reset_savedata()
        

    #save all in one file
    savefile = f'RankNet-MLP_savedata_multilayers_b1_s{loopcnt}_t0.pickle'
    with open(savefile, 'wb') as f: 
        pickle.dump(alldata, f, pickle.HIGHEST_PROTOCOL)
        
        
    acc = []
    for i in ret2.keys():
        df = ret2[i][0]
        _x = stint.get_evalret_shortterm(df)
        acc.append(_x)

    b = np.array(acc)
    print(np.mean(b, axis=0))
    
    #save keys
    #stint._pitmodel.save_keys('pitmodel-keys.pickle')
    
    return b, ret2

In [22]:
def test_pitmodel(modelfile, testevent, runts, loopcnt=10, model='oracle', inlapmode=0,featuremode = stint.FEATURE_PITAGE):
    #pitmodel-m60-mlp-e500-l10-10-5-student-d0.1.pickle
    
    lapmode = {0:'noinlap', 1:'inlap',2:'outlap'}
    
    #dataset = 'indy2013-2018%s-nocarid-context40'%('' if inlapmode==0 else '-inlapstatus')
    dataset = 'indy2013-2018-%s-%s-nocarid-context40'%(
                lapmode[inlapmode],
                'pitage' if featuremode==stint.FEATURE_PITAGE else 'nopitage'
                )
    try:
        if runts == 'RANK':
            acc, ret = simulation(dataset, testevent, 
                        'rank',stint.COL_RANK,'rank',
                       2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                          pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode)        
        else:
            acc, ret = simulation(dataset, testevent, 
                            'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                           2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                              pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode)
    except:
        acc, ret = [],[]
        
    #print(np.mean(acc, axis=0))
    return acc, ret

In [16]:
def plot_prob_forecasts_ex(ts_entry, forecast_entry, ts2, outputfile):

    plot_length = int(forecast_entry[0].samples.shape[1] *1.2) 
    prediction_intervals = (50.0, 90.0)
    legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

    figcnt = len(ts_entry)
    for idx in range(figcnt):
        fig, axs = plt.subplots(1, 1, figsize=(20, 7))
        #ts_entry[idx][-plot_length:].plot(ax=axs)  # plot the time series
        #forecast_entry[idx].plot(prediction_intervals=prediction_intervals, color='g')
        ts_entry[idx].iloc[-plot_length:,0].plot(ax=axs, linewidth=1)  # plot the time series
        
        ts2.iloc[-plot_length:,0].plot(ax=axs, linewidth=1, label='CurRank')  # plot the time series
        
        forecast_entry[idx].copy_dim(0).plot(prediction_intervals=prediction_intervals, color='g')
        
        plt.ylabel('Rank position')
        plt.xlabel('Lap')
        
        laps = range(1, 200, 20)
        
        #axs.set_xticks(laps)
        #axs.set_xticklabels([1+x*30 for x in range(7)])
        #axs.set_xticklabels(laps)
        
        plt.grid(which="both")
        plt.legend(legend, loc="upper left")
        plt.savefig(outputfile + '-%d.pdf'%idx)
        plt.show()

In [17]:
def prisk(full_samples, full_tss, verbose = False):
    carlist = full_tss.keys()
    tss = []
    forecasts = []
    forecasts_mean = []
    freq = '1min'
    start = pd.Timestamp("01-01-2019", freq=freq) 

    for car in carlist:
        testcar = car
        fc = SampleForecast(samples = full_samples[testcar][:, 12:], freq=freq, start_date=start + 12)

        samples = np.mean(full_samples[testcar][:, 12:], axis =0, keepdims=True)
        fc_mean = SampleForecast(samples = samples, freq=freq, start_date=start + 12)

        index = pd.date_range(start='2019-01-01 00:00:00', freq = 'T', periods = len(full_tss[testcar]))
        ts = pd.DataFrame(index = index, data = full_tss[testcar])    

        tss.append(ts)
        forecasts.append(fc)
        forecasts_mean.append(fc_mean)

    evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9]) 
    agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(tss))
    if verbose:
        print(json.dumps(agg_metrics, indent=4))  
    
    print(agg_metrics["wQuantileLoss[0.1]"], agg_metrics["wQuantileLoss[0.5]"],agg_metrics["wQuantileLoss[0.9]"])
    
    return agg_metrics

### fast test

In [18]:
acc, ret, pret = {}, {}, {}

In [25]:
fmodestr = {stint.FEATURE_PITAGE:'pitage',stint.FEATURE_STATUS:'nopitage'}
lapstr = {0:'noinlap',1:'inlap',2:'outlap'}
year = 2018
lapmode = 1
fmode = stint.FEATURE_STATUS
runts = 'RANK'
mid = 'pitmodel-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle', 
                                'Indy500-%s'%year, runts, loopcnt=100,inlapmode=lapmode,featuremode = fmode)


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel_savedata.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel_savedata.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model


model: acc={1.00}, mae={0.00}, rmse={0.00},r2={1.00}, {188}
            naive: acc={1.00}, mae={0.00}, rmse={0.00},r2={1.00}
model: acc={1.00}, mae={0.00}, rmse={0.00},r2={1.00}, {188}
            naive: acc={1.00}, mae={0.00}, rmse={0.00},r2={1.00}
model: acc={1.00}, mae={0.00}, rmse={0.00},r2={1.00}, {188}
            naive: acc={1.00}, mae={0.00}, rmse={0.00},r2={1.00}
model: acc={1.00}, mae={0.00}, rmse={0.00},r2={1.00}, {188}
            naive: acc={1.00}, mae={0.00}, rmse={0.00},r2={1.00}
model: acc={1.00}, mae={0.00}, rmse={0.00},r2={1.00}, {188}
            naive: acc={1.00}, mae={0.00}, rmse={0.00},r2={1.00}
model: acc={1.00}, mae={0.00}, rmse={0.00},r2={1.00}, {188}
            naive: acc={1.00}, mae={0.00}, rmse={0.00},r2={1.00}
model: acc={1.00}, mae={0.00}, rmse={0.00},r2={1.00}, {188}
            naive: acc={1.00}, mae={0.00}, rmse={0.00},r2={1.00}
model: acc={1.00}, mae={0.00}, rmse={0.00},r2={1.00}, {188}
            naive: acc={1.00}, mae={0.00}, rmse={0.00},r2={1.00}


In [8]:
#oracle best: 
#oracle-TIMEDIFF-{year}-inlap-nopitage  ; best mae
#oracle-RANK-{year}-inlap-pitage        ; best top1acc
    
fmodestr = {stint.FEATURE_PITAGE:'pitage',stint.FEATURE_STATUS:'nopitage'}
lapstr = {0:'noinlap',1:'inlap',2:'outlap'}
#for runts in ['TIMEDIFF','RANK']:
#    for fmode in [stint.FEATURE_PITAGE, stint.FEATURE_STATUS]:
for year in ['2018','2019']:
#            for lapmode in [1]:
    lapmode = 1
    fmode = stint.FEATURE_STATUS
    runts = 'TIMEDIFF'
    mid = 'oracle-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
    acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-%s'%year, runts, loopcnt=1,inlapmode=lapmode,featuremode = fmode)
    if acc[mid] != []:
        pret[mid] = prisk(ret[mid][0][1], ret[mid][0][2])
    else:
        pret[mid] = []

    fmode = stint.FEATURE_PITAGE
    runts = 'RANK'
    mid = 'oracle-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
    acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-%s'%year, runts, loopcnt=1,inlapmode=lapmode,featuremode = fmode)
    if acc[mid] != []:
        pret[mid] = prisk(ret[mid][0][1], ret[mid][0][2])
    else:
        pret[mid] = []
    
    

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  app.launch_new_instance()
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  # This is added back by InteractiveShellApp.init_path()
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  
Running evaluation:   0%|          | 0/33 [00:00<?, ?it/s]

model: acc={0.83}, mae={0.88}, rmse={2.11},r2={0.94}, 
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
[[0.82978723 0.882397   2.11167612 0.93774259]
 [0.71808511 1.33932584 3.24896987 0.85262365]]


Running evaluation: 100%|██████████| 33/33 [00:00<00:00, 172.38it/s]
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0


...

init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  # This is added back by InteractiveShellApp.init_path()
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  
Running evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

model: acc={0.89}, mae={0.90}, rmse={2.36},r2={0.93}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
[[0.89361702 0.8953633  2.36438909 0.92699119]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


Running evaluation: 100%|██████████| 32/32 [00:00<00:00, 178.82it/s]

0.03520099673861263 0.06140447310882285 0.034519403422624495


In [9]:
#pitmodel-RANK-{year}-inlap-nopitage    ; best mae & top1acc (pitage a little bit less)
for year in ['2018','2019']:
#            for lapmode in [1]:
    lapmode = 1
    fmode = stint.FEATURE_STATUS
    runts = 'RANK'
    mid = 'pitmodel-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
    acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle', 
                                    'Indy500-%s'%year, runts, loopcnt=3,inlapmode=lapmode,featuremode = fmode)
    if acc[mid] != []:
        pret[mid] = prisk(ret[mid][0][1], ret[mid][0][2])
    else:
        pret[mid] = []

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pitmodel: startlap=10, laps_instint=11, cuation_laps=0,                     nextpos=31
pitmodel: startlap=31, laps_instint=0, cuation_laps=0,                     nextpos=60
pitmodel: startlap=60, laps_instint=0, cuation_laps=0,                     nextpos=90
pitmodel: startlap=90, laps_instint=0, cuation_laps=0,                     nextpos=117
pitmodel: startlap=117, laps_instint=0, cuation_laps=0,                     nextpos=146
pitmodel: startlap=146, laps_instint=0, cuation_laps=0,                     nextpos=178
pitmodel: startlap=178, laps_instint=0, cuation_laps=0,                     nextpos=209
pitmodel: startlap=11, laps_instint=12, cuation_laps=0,                     nextpos=28
pitmodel: startlap=28, 

pitmodel: startlap=23, laps_instint=24, cuation_laps=0,                     nextpos=30
pitmodel: startlap=30, laps_instint=0, cuation_laps=0,                     nextpos=58
pitmodel: startlap=58, laps_instint=0, cuation_laps=0,                     nextpos=89
pitmodel: startlap=89, laps_instint=0, cuation_laps=0,                     nextpos=118
pitmodel: startlap=118, laps_instint=0, cuation_laps=0,                     nextpos=143
pitmodel: startlap=143, laps_instint=0, cuation_laps=0,                     nextpos=171
pitmodel: startlap=171, laps_instint=0, cuation_laps=0,                     nextpos=198
pitmodel: startlap=198, laps_instint=0, cuation_laps=0,                     nextpos=227
pitmodel: startlap=24, laps_instint=25, cuation_laps=0,                     nextpos=31
pitmodel: startlap=31, laps_instint=0, cuation_laps=0,                     nextpos=57
pitmodel: startlap=57, laps_instint=0, cuation_laps=0,                     nextpos=83
pitmodel: startlap=83, laps_instint=0, cuat

pitmodel: startlap=37, laps_instint=6, cuation_laps=0,                     nextpos=61
pitmodel: startlap=61, laps_instint=0, cuation_laps=0,                     nextpos=89
pitmodel: startlap=89, laps_instint=0, cuation_laps=0,                     nextpos=114
pitmodel: startlap=114, laps_instint=0, cuation_laps=0,                     nextpos=146
pitmodel: startlap=146, laps_instint=0, cuation_laps=0,                     nextpos=171
pitmodel: startlap=171, laps_instint=0, cuation_laps=0,                     nextpos=205
pitmodel: startlap=38, laps_instint=7, cuation_laps=0,                     nextpos=69
pitmodel: startlap=69, laps_instint=0, cuation_laps=0,                     nextpos=104
pitmodel: startlap=104, laps_instint=0, cuation_laps=0,                     nextpos=132
pitmodel: startlap=132, laps_instint=0, cuation_laps=0,                     nextpos=163
pitmodel: startlap=163, laps_instint=0, cuation_laps=0,                     nextpos=189
pitmodel: startlap=189, laps_instint=0, 

pitmodel: startlap=53, laps_instint=4, cuation_laps=4,                     nextpos=81
pitmodel: startlap=81, laps_instint=0, cuation_laps=0,                     nextpos=111
pitmodel: startlap=111, laps_instint=0, cuation_laps=0,                     nextpos=147
pitmodel: startlap=147, laps_instint=0, cuation_laps=0,                     nextpos=176
pitmodel: startlap=176, laps_instint=0, cuation_laps=0,                     nextpos=206
pitmodel: startlap=54, laps_instint=5, cuation_laps=4,                     nextpos=86
pitmodel: startlap=86, laps_instint=0, cuation_laps=0,                     nextpos=116
pitmodel: startlap=116, laps_instint=0, cuation_laps=0,                     nextpos=147
pitmodel: startlap=147, laps_instint=0, cuation_laps=0,                     nextpos=181
pitmodel: startlap=181, laps_instint=0, cuation_laps=0,                     nextpos=206
pitmodel: startlap=55, laps_instint=6, cuation_laps=4,                     nextpos=85
pitmodel: startlap=85, laps_instint=0, c

...

pitmodel: startlap=157, laps_instint=22, cuation_laps=11,                     nextpos=172
pitmodel: startlap=172, laps_instint=0, cuation_laps=0,                     nextpos=201
pitmodel: startlap=158, laps_instint=23, cuation_laps=11,                     nextpos=173
pitmodel: startlap=173, laps_instint=0, cuation_laps=0,                     nextpos=203
pitmodel: startlap=159, laps_instint=24, cuation_laps=11,                     nextpos=171
pitmodel: startlap=171, laps_instint=0, cuation_laps=0,                     nextpos=203
pitmodel: startlap=160, laps_instint=25, cuation_laps=11,                     nextpos=173
pitmodel: startlap=173, laps_instint=0, cuation_laps=0,                     nextpos=196
pitmodel: startlap=196, laps_instint=0, cuation_laps=0,                     nextpos=227
pitmodel: startlap=161, laps_instint=26, cuation_laps=11,                     nextpos=172
pitmodel: startlap=172, laps_instint=0, cuation_laps=0,                     nextpos=202
pitmodel: startlap=162

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  # Remove the CWD from sys.path while we load stuff.
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  # This is added back by InteractiveShellApp.init_path()
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  
Running evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

model: acc={0.80}, mae={1.08}, rmse={2.78},r2={0.90}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
model: acc={0.80}, mae={1.10}, rmse={2.81},r2={0.90}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
[[0.79964539 1.09374075 2.79520917 0.89795904]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


Running evaluation: 100%|██████████| 32/32 [00:00<00:00, 179.21it/s]

0.04264722049177324 0.07200703580197149 0.044670013558576714


In [10]:
acc

{'oracle-TIMEDIFF-2018-inlap-nopitage': array([[[0.82978723, 0.882397  , 2.11167612, 0.93774259],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-RANK-2018-inlap-pitage': array([[[0.87765957, 1.02209738, 2.4891525 , 0.91349535],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-TIMEDIFF-2019-inlap-nopitage': array([[[0.81382979, 0.83318529, 2.08506264, 0.94322259],
         [0.73404255, 1.16255107, 3.05012312, 0.87850116]]]),
 'oracle-RANK-2019-inlap-pitage': array([[[0.89361702, 0.8953633 , 2.36438909, 0.92699119],
         [0.73404255, 1.16255107, 3.05012312, 0.87850116]]]),
 'oracle-RANK-2018-inlap-nopitage': array([[[0.7606383 , 1.2411985 , 2.93557541, 0.87968413],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]],
 
        [[0.7712766 , 1.22284644, 2.87999688, 0.88419682],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]],
 
        [[0.77659574, 1.23820225, 2.94022853, 0.87930241],
         [0.71808511, 1.33932584, 3.

In [ ]:
sys.exit(0)

### test low mode

In [11]:
acc, ret, pret = {}, {}, {}

In [12]:
fmodestr = {stint.FEATURE_PITAGE:'pitage',stint.FEATURE_STATUS:'nopitage'}
lapstr = {0:'noinlap',1:'inlap',2:'outlap'}
for runts in ['TIMEDIFF','RANK']:
    for fmode in [stint.FEATURE_PITAGE, stint.FEATURE_STATUS]:
        for year in ['2018','2019']:
            for lapmode in [0,1,2]:
                mid = 'oracle-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
                acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-%s'%year, runts, loopcnt=1,inlapmode=lapmode,featuremode = fmode)
                if acc[mid] != []:
                    pret[mid] = prisk(ret[mid][0][1], ret[mid][0][2])
                else:
                    pret[mid] = []


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p0.pickle with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if __name__ == '__main__':
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  # This is added back by InteractiveShellApp.init_path()
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  
Running evaluation:   0%|          | 0/33 [00:00<?, ?it/s]

model: acc={0.84}, mae={0.94}, rmse={2.18},r2={0.93}, 
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
[[0.83510638 0.93932584 2.17501991 0.93395151]
 [0.71808511 1.33932584 3.24896987 0.85262365]]


Running evaluation: 100%|██████████| 33/33 [00:00<00:00, 176.25it/s]
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0


...

init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p2.pickle with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if __name__ == '__main__':
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  # This is added back by InteractiveShellApp.init_path()
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  
Running evaluation:   0%|          | 0/32 [00:00<?, ?it/s]

model: acc={0.83}, mae={1.11}, rmse={2.83},r2={0.90}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
[[0.82978723 1.11067685 2.82735917 0.89560027]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


Running evaluation: 100%|██████████| 32/32 [00:00<00:00, 177.94it/s]

0.045720497880708956 0.07370491162495267 0.032904589150695635


In [13]:
acc

{'oracle-TIMEDIFF-2018-noinlap-pitage': array([[[0.83510638, 0.93932584, 2.17501991, 0.93395151],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-TIMEDIFF-2018-inlap-pitage': array([[[0.82446809, 0.95992509, 2.2151166 , 0.93149384],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-TIMEDIFF-2018-outlap-pitage': [],
 'oracle-TIMEDIFF-2019-noinlap-pitage': array([[[0.81914894, 0.90460117, 2.18727079, 0.93751979],
         [0.73404255, 1.16255107, 3.05012312, 0.87850116]]]),
 'oracle-TIMEDIFF-2019-inlap-pitage': array([[[0.82978723, 0.97388524, 2.31840941, 0.92980314],
         [0.73404255, 1.16255107, 3.05012312, 0.87850116]]]),
 'oracle-TIMEDIFF-2019-outlap-pitage': [],
 'oracle-TIMEDIFF-2018-noinlap-nopitage': array([[[0.82978723, 0.81797753, 1.97028487, 0.9458006 ],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-TIMEDIFF-2018-inlap-nopitage': array([[[0.83510638, 0.89101124, 2.13030567, 0.93663925],
         [0.718085

In [14]:
fmodestr = {stint.FEATURE_PITAGE:'pitage',stint.FEATURE_STATUS:'nopitage'}
lapstr = {0:'noinlap',1:'inlap',2:'outlap'}
for runts in ['TIMEDIFF','RANK']:
    for fmode in [stint.FEATURE_PITAGE, stint.FEATURE_STATUS]:
        for year in ['2018','2019']:
            for lapmode in [0,1,2]:
                mid = 'pitmodel-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
                acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle', 
                                    'Indy500-%s'%year, runts, loopcnt=3,inlapmode=lapmode,featuremode = fmode)
                if acc[mid] != []:
                    pret[mid] = prisk(ret[mid][0][1], ret[mid][0][2])
                else:
                    pret[mid] = []

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p0.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model
ERROR: key 14-46 not found in model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  # Remove the CWD from sys.path while we load stuff.
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  # This is added back by InteractiveShellApp.init_path()
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  
Running evaluation:   0%|          | 0/33 [00:00<?, ?it/s]

model: acc={0.74}, mae={1.52}, rmse={3.42},r2={0.84}, 
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
model: acc={0.72}, mae={1.52}, rmse={3.40},r2={0.84}, 
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
model: acc={0.73}, mae={1.51}, rmse={3.40},r2={0.84}, 
            naive: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}
[[0.73404255 1.51548065 3.40287593 0.83832909]
 [0.71808511 1.33932584 3.24896987 0.85262365]]


Running evaluation: 100%|██████████| 33/33 [00:00<00:00, 174.73it/s]
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0


...

model: acc={0.80}, mae={1.09}, rmse={2.77},r2={0.90}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
model: acc={0.76}, mae={1.09}, rmse={2.79},r2={0.90}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
model: acc={0.80}, mae={1.09}, rmse={2.79},r2={0.90}, 
            naive: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}
[[0.78723404 1.08722686 2.7839607  0.89877881]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


Running evaluation: 100%|██████████| 32/32 [00:00<00:00, 176.26it/s]
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0


0.041914324453076336 0.07215361500971088 0.04576691462982642


INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p2.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1-1k.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4


In [ ]:
sys.exit(0)

In [15]:
acc

{'oracle-TIMEDIFF-2018-noinlap-pitage': array([[[0.83510638, 0.93932584, 2.17501991, 0.93395151],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-TIMEDIFF-2018-inlap-pitage': array([[[0.82446809, 0.95992509, 2.2151166 , 0.93149384],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-TIMEDIFF-2018-outlap-pitage': [],
 'oracle-TIMEDIFF-2019-noinlap-pitage': array([[[0.81914894, 0.90460117, 2.18727079, 0.93751979],
         [0.73404255, 1.16255107, 3.05012312, 0.87850116]]]),
 'oracle-TIMEDIFF-2019-inlap-pitage': array([[[0.82978723, 0.97388524, 2.31840941, 0.92980314],
         [0.73404255, 1.16255107, 3.05012312, 0.87850116]]]),
 'oracle-TIMEDIFF-2019-outlap-pitage': [],
 'oracle-TIMEDIFF-2018-noinlap-nopitage': array([[[0.82978723, 0.81797753, 1.97028487, 0.9458006 ],
         [0.71808511, 1.33932584, 3.24896987, 0.85262365]]]),
 'oracle-TIMEDIFF-2018-inlap-nopitage': array([[[0.83510638, 0.89101124, 2.13030567, 0.93663925],
         [0.718085

In [16]:
for year in [2018,2019]:
    print(np.mean(acc[f'pitmodel-RANK-{year}-inlap-nopitage'], axis=0))
    
for year in [2018,2019]:
    print(np.mean(acc[f'pitmodel-RANK-{year}-inlap-pitage'], axis=0))
    

[[0.7606383  1.2463171  2.93475227 0.87975036]
 [0.71808511 1.33932584 3.24896987 0.85262365]]
[[0.78723404 1.08722686 2.7839607  0.89877881]
 [0.73404255 1.16255107 3.05012312 0.87850116]]
[[0.7606383  1.22808989 2.92953901 0.88017043]
 [0.71808511 1.33932584 3.24896987 0.85262365]]
[[0.7677305  1.09989933 2.82573003 0.89571164]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


In [17]:
for year in [2018,2019]:
    print(np.mean(acc[f'pitmodel-TIMEDIFF-{year}-inlap-nopitage'], axis=0))
    
for year in [2018,2019]:
    print(np.mean(acc[f'pitmodel-TIMEDIFF-{year}-inlap-pitage'], axis=0))

[[0.7464539  1.40986267 3.23993678 0.85340278]
 [0.71808511 1.33932584 3.24896987 0.85262365]]
[[0.71985816 1.18019779 2.83777545 0.89480835]
 [0.73404255 1.16255107 3.05012312 0.87850116]]
[[0.74822695 1.49712859 3.3041257  0.84755493]
 [0.71808511 1.33932584 3.24896987 0.85262365]]
[[0.70921986 1.40889442 3.24085014 0.86280957]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


In [18]:
for year in [2018,2019]:
    print(np.mean(acc[f'oracle-RANK-{year}-inlap-nopitage'], axis=0))
    
for year in [2018,2019]:
    print(np.mean(acc[f'oracle-RANK-{year}-inlap-pitage'], axis=0))    

[[0.86702128 1.10449438 2.65479567 0.9015992 ]
 [0.71808511 1.33932584 3.24896987 0.85262365]]
[[0.84042553 0.9813466  2.56276687 0.91422601]
 [0.73404255 1.16255107 3.05012312 0.87850116]]
[[0.84574468 0.99850187 2.44152589 0.91677398]
 [0.71808511 1.33932584 3.24896987 0.85262365]]
[[0.88297872 0.89145497 2.35377103 0.92764545]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


In [19]:
for year in [2018,2019]:
    print(np.mean(acc[f'oracle-TIMEDIFF-{year}-inlap-nopitage'], axis=0))
    
for year in [2018,2019]:
    print(np.mean(acc[f'oracle-TIMEDIFF-{year}-inlap-pitage'], axis=0)) 

[[0.83510638 0.89101124 2.13030567 0.93663925]
 [0.71808511 1.33932584 3.24896987 0.85262365]]
[[0.80851064 0.83567241 2.10954231 0.94188157]
 [0.73404255 1.16255107 3.05012312 0.87850116]]
[[0.82446809 0.95992509 2.2151166  0.93149384]
 [0.71808511 1.33932584 3.24896987 0.85262365]]
[[0.82978723 0.97388524 2.31840941 0.92980314]
 [0.73404255 1.16255107 3.05012312 0.87850116]]


In [31]:
import pickle
def save_result(dfs, datafile):
    with open(datafile, 'wb') as f:
        #pack [global_carids, laptime_data]
        savedata = [dfs]
        #savedata = [freq, train_set, test_set]
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)

In [33]:
dfs={}
            
for year in ['2018','2019']:
    mid=f'oracle-TIMEDIFF-{year}-inlap-nopitage'
    mdf, samples, tss = ret[mid][0][0], ret[mid][0][1], ret[mid][0][2]            
    
    if year not in dfs:
        dfs[year] = {}
    dfs[year][mid] = mdf    
    

_trim = 0
_include_final = True
_include_stintlen = True
#_include_stintlen = False
include_str = '1' if _include_final else '0'
stint_str = '1' if _include_stintlen else ''            
outfile=f'shortterm-dfout-ranknet-indy500-tr2013_2017-te2018_2019-end{include_str}-normal-t{_trim}.pickle'
save_result(dfs, outfile)

### conclusion

best model:

  oracle : 
  
     oracle-TIMEDIFF-{year}-inlap-nopitage  ; best mae
     oracle-RANK-{year}-inlap-pitage        ; best top1acc
     
  pitmodel:
  
     pitmodel-RANK-{year}-inlap-nopitage    ; best mae & top1acc (pitage a little bit less)

### test pred models

In [29]:
year=2018
mid=f'oracle-TIMEDIFF-{year}-inlap-nopitage'
mdf, samples, tss = ret[mid][0][0], ret[mid][0][1], ret[mid][0][2]
samples[1].shape


(100, 200)

In [30]:
mdf

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,1,10,3.0,3.0,0.0,0,3.0,0.0,0
1,3,10,7.0,7.0,0.0,0,7.0,0.0,0
2,4,10,12.0,12.0,0.0,0,12.0,0.0,0
3,6,10,15.0,15.0,0.0,0,15.0,0.0,0
4,7,10,28.0,28.0,0.0,0,28.0,0.0,0
...,...,...,...,...,...,...,...,...,...
5335,60,197,15.0,15.0,0.0,0,16.0,1.0,1
5336,64,197,16.0,16.0,0.0,0,14.0,-2.0,-1
5337,66,197,10.0,10.0,0.0,0,10.0,0.0,0
5338,88,197,13.0,13.0,0.0,0,13.0,0.0,0


In [28]:
samples[13][:,:13]

array([[nan, nan, nan, ..., nan, nan, 11.],
       [nan, nan, nan, ..., nan, nan, 11.],
       [nan, nan, nan, ..., nan, nan, 11.],
       ...,
       [nan, nan, nan, ..., nan, nan, 11.],
       [nan, nan, nan, ..., nan, nan, 12.],
       [nan, nan, nan, ..., nan, nan, 12.]])

In [23]:
tss[13]

array([11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 20., 31., 29., 26., 25., 14., 12., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 12., 12., 12., 12., 13., 13.,
       13., 13., 14., 14., 15., 15., 15., 15., 15., 15., 15., 15., 15.,
       15., 15., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [ ]:
mid = 'oracle-2019'
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
mid = 'deepar-2019'
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
mid = 'pitmodel-2019-test'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'RANK',loopcnt=1)
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
mid = 'currank-2019'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'RANK',loopcnt=1,model='naive')
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
mid = 'pitmodel-2018'
full_samples,full_tss = ret[mid][0][1], ret[mid][0][2]
testcar = 12
freq = '1min'
start = pd.Timestamp("01-01-2019", freq=freq) 
fc = SampleForecast(samples = full_samples[testcar][:, 12:], freq=freq, start_date=start + 12)
index = pd.date_range(start='2019-01-01 00:00:00', freq = 'T', periods = len(full_tss[testcar]))
ts = pd.DataFrame(index = index, data = full_tss[testcar])

tss2 = np.zeros_like(full_tss[testcar])
tss2[2:] = full_tss[testcar][:-2]
index2 = pd.date_range(start='2019-01-01 00:00:00', freq = 'T', periods = len(full_tss[testcar]))

ts2 = pd.DataFrame(index = index2, data = tss2)

plot_prob_forecasts_ex([ts], [fc], ts2,'output-car12')

In [ ]:
%debug

In [ ]:
ret.keys()

In [ ]:
np.mean(acc['pitmodel-RANK-2018-inlap-nopitage'],axis=0)

In [ ]:
np.mean(acc['pitmodel-RANK-2019-inlap-nopitage'],axis=0)

In [ ]:
pret['pitmodel-RANK-2018-inlap-nopitage']

In [ ]:
pret['pitmodel-RANK-2019-inlap-nopitage']

### save result df

In [ ]:
df = ret['oracle-2018'][0][0]
df.to_csv('result-shorterm_l2_2018_oracle.csv')

In [ ]:
df = ret['pitmodel-2018'][0][0]
df.to_csv('result-shorterm_l2_2018_pitmodel.csv')

In [ ]:
df = ret['pitmodel-2018'][0][0]
df.to_csv('result-shorterm_l2_2018_pitmodel.csv')